# Classificação de Imagens de 03 classes de plantas por tipos de doenças, com Transfer Learning

#### Objetivo: Realizar o Teste de Inferência com imagens de testes de classe, para o cenário do Caso de Teste 02.

##### Classes Consideradas:
###### Grape___Black_rot
###### Grape___Esca_(Black_Measles)
###### Grape___healthy


## Consultas:
##### https://keras.io/applications/
##### https://www.tensorflow.org/tutorials/load_data/images
##### https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
##### https://keras.io/api/data_loading/
##### https://datascience.stackexchange.com/questions/81541/how-to-predict-multiple-images-from-folder-in-python

## Leitura do local da base de dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
workdir_path = '/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive' #MODIFICAR!!! #Caminho para o seu workspace
os.chdir(workdir_path)

Mounted at /content/drive


In [ ]:
os.path.abspath(os.getcwd())

'/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive'

In [ ]:
# Base de imagens para treino com total de 4659 imagens (classes de tipos de platas)
trainpath = workdir_path + '/classeuva/Train'

# Base de imagens para validação com total 1169 imagens (classes de tipos de platas)
validpath = workdir_path + '/classeuva/Valid'

# Base de imagens para validação com total de 30 imagens (classes de tipos de platas)
testpath = workdir_path + '/classeuva/Test'


## Leitura de Bibliotecas

In [ ]:
from tensorflow.keras.models import load_model
import argparse
import pickle
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import numpy as np
import logging, os
import sys 
from keras.preprocessing import image
import tensorflow as tf
import math
import operator
from pathlib import Path


## Leitura do modelo do respectivo cenário 06 para o Caso de Teste 02

In [ ]:
# Ler modelo treinado gravado para submeter aos dados de testes
modelosalvo = tf.keras.models.load_model('/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive/Inferência-TestesModelos/Caso de Teste 02/Resnet50-cenário06/Modelo_Salvo_1/Resnet50.h5')

## Carrega as Imagens de Testes Selecionadas

In [ ]:
# Prepara variáveis para imagens do tipo Healphy

Black_Measles_path = testpath +'/Black_Measles/' # Caminho das imagens

imagens_Black_Measles = [] # Inicializa uma lista para carregar as imagens


In [ ]:
# Carrega as imagens para do tipo Grape___Esca_(Black_Measles)
for img in os.listdir(Black_Measles_path):                                     # Cria um loop
        img = os.path.join(Black_Measles_path, img)                            # ? pesquisar o que essas funções fazem
        img = tf.keras.utils.load_img(img, target_size=(224,224))    # coloca a imagem no shape largura 64 altura 64
        img = tf.keras.utils.img_to_array(img)                       # transforma em um array
        img = np.expand_dims(img, axis=0)                            # ? pesuisar o que essas funções fazem
        imagem_normalizada = np.array(img, dtype="float") / 255.0    # normaliza a imagem dividindo por 255
        imagens_Black_Measles.append(imagem_normalziada)                         # carrega a imagem para a lista "imagens_healthy"
        
imagens_Black_Measles = np.vstack(imagens_Black_Measles)                         # ? pesquisar o que essas funções fazem

## Previsão Dados de Teste (classe Black_Measles)

In [ ]:
# faz a previsao das imagens de teste do tipo Black_Measles
dados_previsao_imagens_Black_Measles = []                           # inicializa uma lista para colocar o reultado da previsao das imagens na lista
nome_arquivos = []                                                  # inicializa uma lista para colocar o nome dos arquivos
for arquivo in os.listdir(Black_Measles_path):                      # inicializa um loop (para cada arquivo dentro da pasta "heatlthy")
    pred_result = modelosalvo.predict(imagens_Black_Measles)        # faz a previsao de cada arquivo
    dados_previsao_imagens_Black_Measles.append(pred_result)        # coloca os dados da previsão na lsita "dados_previsao_imagens_healthy"
    nome_arquivos.append(arquivo)                                   # coloca o nome do arquivo na lista "nome_arquivos"

In [ ]:
dados_previsao_imagens_Black_Measles[0]

In [ ]:
# Define os labels R:Grape___Black_rot, M:Grape___Esca_(Black_Measles) e H:Grape___healthy' na mesma ordem de classes do modelo na etapa de load.
# Previsão dados de testes selecionados

label_file_path = os.path.join("RMH", 'labels') 

#sorts attributes according to confidence score (how probable attribute exists)
top_k = []
pred = [] 
for i in range(len(dados_previsao_imagens_Black_Measles)):
            ordem = dados_previsao_imagens_Black_Measles[i][0].argsort()[-len(dados_previsao_imagens_Black_Measles[i][0]):][::-1]
            top_k.append(ordem)
            top = top_k[i][:15]
            print(nome_arquivos[i])
            for node_id in top:
                planta_string = label_file_path[node_id]
                score = dados_previsao_imagens_Black_Measles[i][0][node_id]
                print('%s (score = %.5f)' % (planta_string, score))